# Ömer Seyfettin ile Basit bir WorCount 

In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
# Create SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

spark = SparkSession.builder \
.master("local[4]") \
.appName("RDD-Olusturmak") \
.config("spark.executor.memory","16g") \
.config("spark.driver.memory","4g") \
.getOrCreate()

# sparkContext'i kısaltmada tut
sc = spark.sparkContext

## Veriyi okuma

In [62]:
#veri_dosyasi = "C:\\Users\\toshiba\\SkyDrive\\veribilimi.co\\udemy-apache-spark\\data\\omer_seyfettin_forsa_hikaye.txt"
#veri_dosyasi = "C:\\Users\\toshiba\\SkyDrive\\veribilimi.co\\udemy-apache-spark\\data\\omer_seyfettin_kurbaga_duasi.txt"
veri_dosyasi = "C:\\Users\\toshiba\\SkyDrive\\veribilimi.co\\udemy-apache-spark\\data\\omer_seyfettin_falaka.txt"


hikayeRDD = sc.textFile(veri_dosyasi)

In [46]:
hikayeRDD.take(5)

['Falaka  (Ömer Seyfettin)',
 '2 sene ÖnceYorum yap Süleyman Kara2.422 Görüntüleme',
 'Her sabah Çarşı Camii`in arkasındaki harap zaptiye ahırlarının önünden, bir serçe sürüsü gibi, cıvıl cıvıl neşeli geçerdik. Okul biraz daha ileride, alçak duvarlı, oldukça geniş bir avlunun ortasında idi. Bir kattı, etrafında yükselen büyük kestane ağaçlarının birbirine karışmış koyu gölgeleri bütün çatısını kaplardı. Biz daha avlunun kapısından Hoca girmeden Efendinin olup olmadığını, şöyle bir bakar, anlardık:',
 '',
 '-Abdurrahman Çelebi gelmiş mi be?']

In [22]:
# Her bir kelimeyi boşluklarla ayıralım ve başka bir rdd'de tutalım
kelimeler = hikayeRDD.flatMap(lambda satir: satir.split(" "))

In [23]:
# Kelimeleri sayalım
kelime_sayilari = kelimeler.map(lambda kelime: (kelime,1)).reduceByKey(lambda x,y: x+y)

In [24]:
# kaç farklı kelime var
kelime_sayilari.count()

1056

In [25]:
# Kelimeler ve tekrarlanma sayılarından rastgel 15 tanesini görelim
kelime_sayilari.take(15)

[('', 142),
 ('Seyfettin)', 1),
 ('sene', 1),
 ('ÖnceYorum', 1),
 ('Her', 4),
 ('arkasındaki', 1),
 ('harap', 1),
 ('zaptiye', 1),
 ('ahırlarının', 1),
 ('önünden,', 1),
 ('sürüsü', 1),
 ('gibi,', 3),
 ('cıvıl', 2),
 ('neşeli', 1),
 ('geçerdik.', 1)]

In [26]:
# Rakamları anahtar olarak kullanmak için 0 indisine kelimeleri 1 indisine atalım
kelime_sayilari2 = kelime_sayilari.map(lambda x: (x[1], x[0]))

In [28]:
# Rakamlar artık key olunca saydoralım bakalım en çok tekrarlanan 15 kelime ne imiş
kelime_sayilari2.sortByKey(False).take(20)

[(142, ''),
 (39, '–'),
 (38, 'bir'),
 (34, 'Hoca'),
 (19, 'Efendi'),
 (17, 'gibi'),
 (10, 'diye'),
 (10, 'Abdurrahman'),
 (9, 'bu'),
 (8, 'sonra'),
 (7, 'falaka'),
 (7, 'da'),
 (7, 'Efendinin'),
 (7, 'kadar'),
 (7, 'gün'),
 (6, 'daha'),
 (6, 'yere'),
 (6, 'biri'),
 (6, 'boş'),
 (6, 'ne')]

# Uygulamayı geliştirelim

In [101]:
import re, string

In [103]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [125]:
punctuations = string.punctuation
def clean_word(dirty_word):
    clean_word_utf8 = dirty_word.encode('utf-8')
    clean_word_utf8_lowered = clean_word_utf8.lower()
    clean_word_utf8_lowered = clean_word_utf8_lowered.replace('--',' ')
    clean_word_cleaned = clean_word_utf8_lowered.translate(None,punctuations)
    return clean_word_cleaned

In [126]:
hikayeRDD = sc.textFile(veri_dosyasi)

In [127]:
hikayeRDD.take(5)

['Falaka  (Ömer Seyfettin)',
 '2 sene ÖnceYorum yap Süleyman Kara2.422 Görüntüleme',
 'Her sabah Çarşı Camii`in arkasındaki harap zaptiye ahırlarının önünden, bir serçe sürüsü gibi, cıvıl cıvıl neşeli geçerdik. Okul biraz daha ileride, alçak duvarlı, oldukça geniş bir avlunun ortasında idi. Bir kattı, etrafında yükselen büyük kestane ağaçlarının birbirine karışmış koyu gölgeleri bütün çatısını kaplardı. Biz daha avlunun kapısından Hoca girmeden Efendinin olup olmadığını, şöyle bir bakar, anlardık:',
 '',
 '-Abdurrahman Çelebi gelmiş mi be?']

In [129]:
# Her bir kelimeyi boşluklarla ayıralım ve başka bir rdd'de tutalım
cleaned_word_rdd = hikayeRDD.flatMap(lambda x: clean_word(x).split())
cleaned_word_rdd.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 123.0 failed 1 times, most recent failure: Lost task 0.0 in stage 123.0 (TID 126, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 230, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 225, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\spark\python\pyspark\rdd.py", line 1371, in takeUpToNumLeft
    yield next(iterator)
  File "C:\spark\python\lib\pyspark.zip\pyspark\util.py", line 55, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-129-6349202b2364>", line 2, in <lambda>
  File "<ipython-input-125-813c9dace119>", line 5, in clean_word

In [92]:
# Kelimeleri sayalım
kelime_sayilari = kelimeler.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
kelime_sayilari.take(5)

[('', 142), ('Seyfettin)', 1), ('sene', 1), ('ÖnceYorum', 1), ('Her', 4)]

In [93]:
# Rakamları anahtar olarak kullanmak için 0 indisine kelimeleri 1 indisine atalım
kelime_sayilari2 = kelime_sayilari.map(lambda x: (x[1], x[0]))
kelime_sayilari2.take(5)

[(142, ''), (1, 'Seyfettin)'), (1, 'sene'), (1, 'ÖnceYorum'), (4, 'Her')]

In [94]:
# Rakamlar artık key olunca saydoralım bakalım en çok tekrarlanan 15 kelime ne imiş
kelime_sayilari2.sortByKey(False).take(20)

[(142, ''),
 (39, '–'),
 (38, 'bir'),
 (34, 'Hoca'),
 (19, 'Efendi'),
 (17, 'gibi'),
 (10, 'diye'),
 (10, 'Abdurrahman'),
 (9, 'bu'),
 (8, 'sonra'),
 (7, 'falaka'),
 (7, 'da'),
 (7, 'Efendinin'),
 (7, 'kadar'),
 (7, 'gün'),
 (6, 'daha'),
 (6, 'yere'),
 (6, 'biri'),
 (6, 'boş'),
 (6, 'ne')]

In [97]:
c=[]

In [98]:
c.append(kelime_temizle('sen'))
c

[['s']]

In [100]:
c.append(kelime_temizle('ben'))
c

[['s'], ['b'], ['b']]

In [95]:
temizlenecek_kelimeler = ['','–','bir','gibi','diye','bu','sonra','da','kadar','daha','biri','ne']
def kelime_temizle(kelime):
    sonuc = []
    if(kelime not in temizlenecek_kelimeler):
        sonuc.append(kelime[0])
    return sonuc

In [96]:
kelime_temizle('sen')

['s']

In [137]:
print("Hello şey".encode('utf-8'))

b'Hello \xc5\x9fey'
